# K-Means(Algorytm Centroidów)

Methods to predict the unlabeled data. Often there will be a need to create the groups of data instead of trying to predict classes or values. This sort of problems is called clustring. We can think of it as a way to create clusters. To be more specific there will be cases that will demand to cluster customer data into distinct groups. We can try to cluster some desises into benign and the deadly ones. But there is no guarantee that the cluster will fall into the correct lines. 

K-means clustering - unsupervised learning algorithm - It will attempt to group similair cluster together. 

- Clustering similiar documetns
- Clustering based on features
- Market segmentation
- Identify similair physical groups

There are some particluar steps for K-means clustring: 

- The analyst needs to setup the number of cluster. Our "K" value. It happens before we create the model. That means the analyst needs to use the domain knowledge to choose correct K value. 
- Then the random point assigment happens for each cluster. For each cluster it is going to compute the cluster centroid by taking the mean vectors points of the cluster. 
- After that it will assign each point to the cluster wheter the point the most centered(centroid). 

There are no pros and cons of assining the cluster. There is however the mathematical elbow method. 

Is called the elbow method. First of all we are computing the sum o squared error (SSE) for values of k e.g 2,4,6,8
The SSE is defined as the sum of the squared distance between each member of the cluster and its centroid. 

Then we suppose to plot K against the SSE. The error suppose to decrase as K gets larger. 
The idea behind this is to choose the K at which the SSE decreases abruptly. This produces the elbow effect in the graf. 

While looking at the plot. We need to find where the abroubt values breaks and creates something shaped as the bending arm. 
PySpark as it self does not supply the plotting method but we will be using matplotlib to fulfill this one. 


In [1]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('K-meansClusteringTheory').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 10:00:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/15 10:00:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.ml.clustering import KMeans

In [5]:
dataset = spark.read.format('libsvm').load('sample_kmeans_data.txt')

22/09/15 10:02:37 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [7]:
final_data = dataset.select('features')

In [8]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [33]:
# Setting up the kmeans: 
 # setK represent the K value setting. Here we are following the documentation. 
 # setSeed - the place of centriod is by default randomly choosen. setSeed() works in case we would like to come back to our 
  # project and and have quote to quote random setting. 
    
kmeans = KMeans().setK(3).setSeed(1)

In [34]:
# kmeans only need the features column. The other type of operation would require the label Column as well. 
model = kmeans.fit(final_data)

In [35]:
# Deprecated in 3.0.0
# wssse = model.computeCost(final_data)

In [36]:
# # Solution since PySpark 3.0.0
wssse = model.summary.trainingCost

In [37]:
# The within sum of squared errors
print(wssse)

0.07499999999994544


In [38]:
centers = model.clusterCenters()

In [39]:
centers

[array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [40]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [41]:
results = model.transform(final_data)

In [42]:
results.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

